In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = [224, 224]

train_path = 'dataset/train'
valid_path = 'dataset/test'

In [5]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# don't need to train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [7]:
# useful for getting number of output classes, output classes will be two
folders = glob('dataset/train/*'
)

In [8]:
# adding a layer of flattening
x = Flatten()(vgg.output)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)
#adding the last layer also with the softmax activation function

In [10]:
#creating a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
model.summary()
#checking out the model summary

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# defining the loss , optimizer and metrics. parameters we need to choose
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
#Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
#give the target size same we used in initialization
training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [16]:
import scipy.integrate as integrate
import scipy

In [17]:
# fit the model with 5 epochs. took nearly 2 hours to train in my intel i5, 16gb ram, 2gb nvidia graphics configs
history = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 903s 6s/step - loss: 0.2155 - accuracy: 0.9199 - val_loss: 0.2343 - val_accuracy: 0.9071
Epoch 2/5
163/163 [==============================] - 919s 6s/step - loss: 0.1141 - accuracy: 0.9569 - val_loss: 0.3015 - val_accuracy: 0.9006
Epoch 3/5
163/163 [==============================] - 1061s 7s/step - loss: 0.0906 - accuracy: 0.9688 - val_loss: 0.3561 - val_accuracy: 0.8942
Epoch 4/5
163/163 [==============================] - 1048s 6s/step - loss: 0.0895 - accuracy: 0.9672 - val_loss: 0.2620 - val_accuracy: 0.9151
Epoch 5/5
163/163 [==============================] - 1030s 6s/step - loss: 0.0768 - accuracy: 0.9712 - val_loss: 0.3289 - val_accuracy: 0.9119


In [6]:
import tensorflow as tf
from keras.models import load_model

model.save('model_vgg16_lungdisease.h5')
#saving the model for future 


NameError: name 'model' is not defined

In [1]:
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16
import matplotlib.pyplot as plt
from pathlib import Path
from keras.utils import load_img, img_to_array 

In [2]:
import tensorflow as tf
from keras.models import load_model
model = load_model('model_vgg16_lungdisease.h5')
#loading the saved model to make prediction

In [3]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
#this is not needed as are using a pre trained model

In [34]:
#Image path 
path = Path("n2.jpeg")
#Load the image
img = load_img(path, target_size=(224,224))

#convert image into an array using numpy
image_array = img_to_array(img)


#expanding the dimension because we are giving only one image as input
x_train = np.expand_dims(image_array, axis=0)

#making prediction
pred = model.predict(x_train)


#softmax gives the probabilities of output classes. so the prediction will be the one with high probability
if pred[0][0]>pred[0][1]:
    result="x-ray looks normal , but although i recommend consulting a radiologist for further informations"
else:
    result="""x-ray looks like you are infected with pneumonia. This result is based on a deep learning algorithm trained over 5000+ xray images and might not be 100% true 
      please consult a radiologist"""


1/1 [==============================] - 0s 123ms/step


In [35]:
pred


array([[0., 1.]], dtype=float32)